In [1]:
from tools import *
from tqdm import tqdm
from typing import List, Dict
import json
import requests
from tqdm import tqdm
from typing import List, Dict,Tuple
import json
import requests
import tools
from typing import List
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
import boto3
from langchain_aws import ChatBedrock
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
from langchain_core.output_parsers import StrOutputParser


/home/ec2-user/newenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
stats = tools.clear_all_graph_data()


🗑️  Starting to clear all graph data...
   Deleting all relationships...
   Deleting all nodes...
   Cleaning up GDS graphs...
   - No 'entity' graph to drop
   Dropping all indexes...
   Dropping all constraints...
✅ Graph cleanup completed!
   - Deleted 0 relationships
   - Deleted 0 nodes
   - Remaining nodes: 0


In [3]:
url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)

In [4]:
def chunk_into_books(text: str) -> List[str]:
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [5]:
token_count = [num_tokens_from_string(el) for el in books]
print(
    f"""There are {len(token_count)} books with token sizes:
- avg {sum(token_count) / len(token_count)}
- min {min(token_count)}
- max {max(token_count)}
"""
)

There are 24 books with token sizes:
- avg 6515.208333333333
- min 4459
- max 10760



In [6]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [7]:
#이미 정해져 있는 entity
ENTITY_TYPES = "PERSON,ORGANIZATION,LOCATION,GOD,EVENT,CREATURE,WEAPON_OR_TOOL"

def extract_entities(text: str) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract entities and relationships from text using Bedrock
    
    Args:
        text: Input text to analyze
        
    Returns:
        Tuple of (entities, relationships) as lists of dictionaries
    """
    # Construct prompt
    messages = [
        {"role": "user", "content": tools.create_extraction_prompt(ENTITY_TYPES, text)}
    ]
    
    # Make the LLM call using Bedrock (model parameter is ignored)
    output = tools.chat(messages, model="gpt-4o")  # Will use Bedrock instead
    
    # Parse output and return both entities and relationships
    return tools.parse_extraction_output(output)

In [8]:
final_nodes = [] 
final_relationship = [] 
number_of_books = 1
for book_i, book in enumerate(
    tqdm(chunked_books[:number_of_books], desc="Processing Books")
):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        final_nodes.extend(nodes)
        final_relationship.extend(relationships)

Book 0:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MUSE;GOD;Divine being invoked to tell the story of the ingenious hero who travelled far and wide after sacking Troy)
|
("entity";TROY;LOCATION;Famous town that was sacked by the ingenious hero)
|
("entity";HYPERION;GOD;Sun-god whose cattle were eaten by the hero's men, leading to their destruction)
|
("entity";JOVE;GOD;Father deity, referred to as the father of the Muse)
|
("entity";ULYSSES;PERSON;Ingenious hero who travelled far and wide after sacking Troy, trying to return home to his wife and country)
|
("entity";CALYPSO;GOD;Goddess who detained Ulysses in a large cave and wanted to marry him)
|
("relationship";MUSE;JOVE;The Muse is referred to as the daughter of Jove;8)
|
("relationship";ULYSSES;TROY;Ulysses sacked the famous town of Troy;9)
|
("relationship";ULYSSES;HYPERION;Ulysses's men perished after eating Hyperion's cattle, which prevented their return home;7)
|
("relationship";HYPERION;ULYSSES


Book 0:   5%|▍         | 1/22 [00:07<02:34,  7.35s/it]

 Ulysses from returning to his wife and country;9)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";ITHACA;LOCATION;The homeland where the hero is destined to return after years of wandering)
|
("entity";NEPTUNE;GOD;God who persecuted the hero without ceasing and would not let him get home, later went to the Ethiopians for a festival)
|
("entity";ETHIOPIANS;PERSON;People at the world's end who lie in two halves, one looking West and the other East, who received Neptune for a festival)
|
("entity";OLYMPIAN JOVE;GOD;Sire of gods and men who spoke first when the gods met in his house, also called the father of gods)
|
("entity";AEGISTHUS;PERSON;Man who made love to Agamemnon's wife unrighteously and killed Agamemnon, later killed by Orestes)
|
("entity";AGAMEMNON;PERSON;Man who was killed by Aegisthus, father of Orestes)
|
("entity";ORESTES;PERSON;Son of Agamemnon who killed Aegisthus in revenge for his father's murder)
|
("relationship";NEPTUNE;ITHACA;Nep


Book 0:   9%|▉         | 2/22 [00:15<02:39,  7.99s/it]

...


Book 0:  91%|█████████ | 20/22 [03:38<00:22, 11.26s/it]

)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;Young man who states his father is dead, puts no faith in rumors, knows in his heart the stranger was a goddess, has a room high up in a tower)
|
("entity";ULYSSES;PERSON;Telemachus's dead father who was an old friend of Mentes)
|
("entity";PENELOPE;PERSON;Telemachus's mother who sometimes sends for a soothsayer and questions him)
|
("entity";SOOTHSAYER;PERSON;Person whom Penelope sometimes sends for and questions, whose prophecies Telemachus gives no heed)
|
("entity";MENTES;PERSON;Son of Anchialus, chief of the Taphians, old friend of Ulysses, identity claimed by the stranger)
|
("entity";ANCHIALUS;PERSON;Father of Mentes)
|
("entity";TAPHIANS;PERSON;People ruled by Mentes as their chief)
|
("entity";MINERVA;GOD;Goddess who Telemachus knew in his heart had been the stranger)
|
("entity";SUITORS;PERSON;Men who returned to singing and dancing until evening, then went home to bed in the


Book 0:  95%|█████████▌| 21/22 [03:53<00:12, 12.35s/it]

 room;7)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";LAERTES;PERSON;Man who had a wedded wife and feared her resentment, so did not take Euryclea to his bed despite respecting her)
|
("entity";LAERTES' WIFE;PERSON;Wedded wife of Laertes whose resentment he feared regarding Euryclea)
|
("entity";EURYCLEA;PERSON;Good old woman who lighted Telemachus to his room, loved him better than other women in the house, had nursed him as a baby, folded his shirt and hung it up)
|
("entity";TELEMACHUS;PERSON;Young man who was nursed by Euryclea as a baby, went to his bedroom, took off his shirt, lay covered with woollen fleece thinking of his voyage)
|
("entity";BEDROOM;LOCATION;Telemachus's room where he sat on the bed and slept)
|
("entity";BED;WEAPON_OR_TOOL;Furniture where Telemachus sat and later lay covered with woollen fleece)
|
("entity";SHIRT;WEAPON_OR_TOOL;Garment that Telemachus took off and gave to Euryclea to fold and hang up)
|
("entity";PEG;WEAPON


Processing Books: 100%|██████████| 1/1 [04:07<00:00, 247.43s/it]


In [9]:
import pandas as pd

# DataFrame으로 변환 후 CSV 저장
relationship = pd.DataFrame(final_relationship)
nodes = pd.DataFrame(final_nodes)

relationship.to_csv('relationship.csv', index=False)
nodes.to_csv('nodes.csv', index=False)


In [10]:
# df = pd.read_csv('final_relationship.csv')
# final_relationship = df.to_dict('records')

In [11]:
final_relationship[0]

{'record_type': 'relationship',
 'source_entity': 'MUSE',
 'target_entity': 'JOVE',
 'relationship_description': 'The Muse is referred to as the daughter of Jove',
 'relationship_strength': 8}

In [3]:
from pydantic import BaseModel, Field

class RelationshipVerbResult(BaseModel):
    subject: str
    object: str
    verbs: List[str]

class AllVerbsResult(BaseModel):
    results: List[RelationshipVerbResult]

class SimplifiedSentenceResult(BaseModel):
    """간단한 문장 결과"""
    subject: str = Field(description="주어")
    object: str = Field(description="목적어") 
    verb: str = Field(description="동사")
    simplified_sentence: str = Field(description="간단하게 만든 문장")

In [4]:
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.1,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None
    def extract_specific_relationship_verbs(self, relationships: List[dict]) -> AllVerbsResult:
        """주체가 객체에게 한 행위 동사 추출"""
        all_results = []
        
        for item in relationships:
            result = self.process_row(
                item["relationship_description"], 
                item["source_entity"], 
                item["target_entity"]
            )
            all_results.append(result)
            print(f"Processed: {result.subject} → {result.object}: {result.verbs}")
        
        return AllVerbsResult(results=all_results)

    def process_row(self, description: str, subject: str, object_entity: str) -> RelationshipVerbResult:
        """단일 관계 처리"""
        
        # Structure output parser 설정
        parser = PydanticOutputParser(pydantic_object=RelationshipVerbResult)
        
        # PromptTemplate 생성
        prompt_template = PromptTemplate(
            template="""Extract ALL verbs from this relationship description:
            Description: {description}
            Subject: {subject}, Object: {object_entity}
            
            Extract these types of verbs:
            1. Active verbs (killed, bought, sacked)
            2. Passive verbs (is referred to as, is described as, is called)
            3. Phrasal verbs (take out, look after, stand out)
            4. Compound verbs (is married to, is associated with)
            
            Rules:
            - Include ALL verb forms including "is/was + past participle" constructions
            - Include "is referred to as" → extract "refer"
            - Include "is described as" → extract "describe"
            - Include "is called" → extract "call"
            - Use base form of verbs (refer instead of referred, describe instead of described)
            
            Examples:
            - "is referred to as" → "refer"
            - "is described as" → "describe" 
            - "killed" → "kill"
            - "bought" → "buy"
            
            Always return at least one verb that captures the relationship.
            
            {format_instructions}
            """,
            input_variables=["description", "subject", "object_entity"],
            partial_variables={"format_instructions": parser.get_format_instructions()}
        )
        
        try:
            # Chain 생성 및 실행
            chain = prompt_template | self.llm | parser
            result = chain.invoke({
                "description": description,
                "subject": subject,
                "object_entity": object_entity
            })
            return result
        except Exception as e:
            print(f"❌ 관계 동사 추출 실패: {e}")
            return RelationshipVerbResult(
            subject=subject, 
            object=object_entity, 
            verbs=["relate"]
        )

    def extract_simplified_sentence_verbs(self, relationships: List[dict]) -> List[SimplifiedSentenceResult]:
        """주체가 객체에게 한 행위 동사 추출"""
        all_results = []
        
        for item in relationships:
            result = self.create_simplified_sentence(
                item["relationship_description"], 
                item["source_entity"], 
                item["target_entity"],
                item["verbs"]
            )
            all_results.append(result)
            # verbs → verb로 수정
            print(f"Processed: {result.subject} → {result.object}: {result.verb}")
            print(f"Simplified: {result.simplified_sentence}")
    
        return all_results  # AllVerbsResult 대신 직접 리스트 반환
    
    def create_simplified_sentence(self, description: str, subject: str, object_entity: str, verb: str) -> SimplifiedSentenceResult:
        """verb를 중심으로 간단한 문장 생성"""
        
        # Structure output parser 설정
        parser = PydanticOutputParser(pydantic_object=SimplifiedSentenceResult)
        
        # PromptTemplate 생성
        prompt_template = PromptTemplate(
            template="""Create a simple sentence using only the essential information:
            
            Original description: {description}
            Subject: {subject}
            Object: {object_entity}
            Focus verb: {verb}
            
            Rules:
            1. Create ONE simple sentence using the format: [Subject] [Verb] [Object]
            2. Remove all unnecessary details, descriptions, and context
            3. Keep only information directly related to the verb action
            4. Use active voice when possible
            5. Remove phrases like "is referred to as", "is described as" - just use the core verb
            
            Examples:
            - "The Muse is referred to as the daughter of Jove" + verb "refer" → "Jove refers to Muse as daughter"
            - "Achilles killed Hector in battle" + verb "kill" → "Achilles killed Hector"
            - "Paris is married to Helen" + verb "marry" → "Paris married Helen"
            
            {format_instructions}
            """,
            input_variables=["description", "subject", "object_entity", "verb"],
            partial_variables={"format_instructions": parser.get_format_instructions()}
        )
        
        try:
            # Chain 생성 및 실행
            chain = prompt_template | self.llm | parser
            result = chain.invoke({
                "description": description,
                "subject": subject,
                "object_entity": object_entity,
                "verb": verb
            })
            return result
        except Exception as e:
            print(f"❌ 문장 생성 실패: {e}")
            return SimplifiedSentenceResult(
                subject=subject,
                object=object_entity,
                verb=verb,
                simplified_sentence=f"{subject} {verb} {object_entity}"
            )
            


In [5]:
llm = AdvancedLLM()


In [6]:
len(final_relationship)

NameError: name 'final_relationship' is not defined

In [ ]:
result = llm.extract_specific_relationship_verbs(final_relationship)


# 결과를 DataFrame으로 변환
data = []
for item in result.results:
    data.append({
        'subject': item.subject,
        'object': item.object,
        'verbs': ', '.join(item.verbs)  # 리스트를 문자열로 변환
    })

df = pd.DataFrame(data)

# CSV로 저장
df.to_csv('preprocessing_relationship.csv', index=False)

print(f"Saved {len(df)} relationships to relationship_verbs.csv")

Processed: MUSE → JOVE: ['refer']
Processed: ULYSSES → TROY: ['sack']
Processed: ULYSSES → HYPERION: ['perish', 'eat', 'prevent', 'return']
Processed: HYPERION → ULYSSES: ['prevent', 'reach', 'eat']
Processed: ULYSSES → CALYPSO: ['detain', 'want', 'marry']
Processed: CALYPSO → ULYSSES: ['keep', 'return']
Processed: NEPTUNE → ITHACA: ['prevent', 'reach']
Processed: NEPTUNE → ETHIOPIANS: ['go', 'accept', 'enjoy']
Processed: OLYMPIAN JOVE → AEGISTHUS: ['think', 'speak']
Processed: AEGISTHUS → AGAMEMNON: ['kill', 'make']
Processed: ORESTES → AEGISTHUS: ['kill']
Processed: ORESTES → AGAMEMNON: ['be']
Processed: AEGISTHUS → AGAMEMNON: ['make']
Processed: MERCURY → AEGISTHUS: ['send', 'warn', 'kill', 'make']
Processed: ORESTES → AEGISTHUS: ['take', 'grow']
Processed: MINERVA → SATURN: ['be']
Processed: MINERVA → ULYSSES: ['bleed']
Processed: ATLAS'S DAUGHTER → ATLAS: ['be']
Processed: ATLAS'S DAUGHTER → ULYSSES: ['capture', 'try', 'make', 'forget']


In [27]:
verbs=pd.read_csv("preprocessing_relationship.csv" )
relationtest= pd.read_csv("relationship.csv" )

In [34]:
data=relationtest[["source_entity", "target_entity" , "relationship_description" ]]

In [39]:
data["verbs"]= verbs["verbs"]

/tmp/ipykernel_3565872/643482045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["verbs"]= verbs["verbs"]


In [41]:
data['verbs'] = data['verbs'].str.split(', ')
data_exploded = data.explode('verbs')

/tmp/ipykernel_3565872/2352784206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['verbs'] = data['verbs'].str.split(', ')


In [43]:
data_exploded.to_csv("verb_explore.csv", index=False)

In [7]:
ff=pd.read_csv("verb_explore.csv")

In [8]:
regenerate_relationship = ff.to_dict('records')

In [9]:
result=llm.extract_simplified_sentence_verbs(regenerate_relationship)

Processed: Muse → Jove: is
Simplified: Muse is Jove's daughter
Processed: Jove → Muse: refers
Simplified: Jove refers to Muse as daughter
Processed: Ulysses → Troy: sacked
Simplified: Ulysses sacked Troy
Processed: Ulysses → Hyperion: perished
Simplified: Ulysses perished through Hyperion
Processed: Ulysses → Hyperion's cattle: ate
Simplified: Ulysses ate Hyperion's cattle
Processed: Hyperion → Ulysses: prevented
Simplified: Hyperion prevented Ulysses
Processed: Ulysses → Hyperion: return
Simplified: Ulysses returned to Hyperion
Processed: Hyperion → Ulysses: prevented
Simplified: Hyperion prevented Ulysses
Processed: Hyperion → Ulysses: prevented
Simplified: Hyperion prevented Ulysses from reaching home
Processed: Ulysses's men → cattle: ate
Simplified: Ulysses's men ate cattle
Processed: Calypso → Ulysses: detained
Simplified: Calypso detained Ulysses
Processed: Calypso → Ulysses: wanted
Simplified: Calypso wanted Ulysses
Processed: CALYPSO → ULYSSES: marry
Simplified: Calypso marrie

In [10]:
result

[SimplifiedSentenceResult(subject='Muse', object='Jove', verb='is', simplified_sentence="Muse is Jove's daughter"),
 SimplifiedSentenceResult(subject='Jove', object='Muse', verb='refers', simplified_sentence='Jove refers to Muse as daughter'),
 SimplifiedSentenceResult(subject='Ulysses', object='Troy', verb='sacked', simplified_sentence='Ulysses sacked Troy'),
 SimplifiedSentenceResult(subject='Ulysses', object='Hyperion', verb='perished', simplified_sentence='Ulysses perished through Hyperion'),
 SimplifiedSentenceResult(subject='Ulysses', object="Hyperion's cattle", verb='ate', simplified_sentence="Ulysses ate Hyperion's cattle"),
 SimplifiedSentenceResult(subject='Hyperion', object='Ulysses', verb='prevented', simplified_sentence='Hyperion prevented Ulysses'),
 SimplifiedSentenceResult(subject='Ulysses', object='Hyperion', verb='return', simplified_sentence='Ulysses returned to Hyperion'),
 SimplifiedSentenceResult(subject='Hyperion', object='Ulysses', verb='prevented', simplified_s